In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
Comparison_model = torch.load('model/singleComparison.pkl')
Contingency_model = torch.load('model/singleContingency.pkl')
Expansion_model = torch.load('model/singleExpansion.pkl')
Temporal_model = torch.load('model/singleTemporal.pkl')

In [58]:
def load_PDTB(type, word_cnt={}):
    with open(config.resourses.data_base_dir + "im"+ type +"Corenlp", "r") as f:
        data = json.load(f)
    
    stop_words = []
    max_seq_len = config.model.seq_len

    arg1_sents = []
    arg2_sents = []
    labels = []

    for _ ,value in data.items():
        label = value['Sense'].split('.')[0]
        
        if label == "Comparison":
            label = 0
        elif label == "Contingency":
            label = 1
        elif label == "Expansion":
            label = 2
        elif label == "Temporal":
            label = 3
        else:
            continue
        
        
        arg1_words = []
        for word in value['Arg1']['Tokens']:
            if not stop_words or word['Word'] not in stop_words:
                arg1_words.append(word['Word'])
                if not word_cnt.get(word['Word']):
                    word_cnt[word['Word']] = 0
                word_cnt[word['Word']] += 1

        arg2_words = []
        for word in value['Arg2']['Tokens']:
            if not stop_words or word['Word'] not in stop_words:
                arg2_words.append(word['Word'])
                if not word_cnt.get(word['Word']):
                    word_cnt[word['Word']] = 0
                word_cnt[word['Word']] += 1


        if len(arg1_words) < max_seq_len and len(arg2_words) < max_seq_len:
            arg1_sents.append(arg1_words)
            arg2_sents.append(arg2_words)
            labels.append(label)
        # loss samples: train(432)  test(34)  dev(36)

    return arg1_sents, arg2_sents, labels, word_cnt

In [4]:
def build_up_word_dict():
    word_to_id = {'OOV':0}
    num_words = 1
    word_cnt = {}
    _, _, _, word_cnt = load_PDTB("Train", word_cnt)
    _, _, _, word_cnt = load_PDTB("Test", word_cnt)
    _, _, _, word_cnt = load_PDTB("Dev", word_cnt)

    # first 10k words
    word_cnt = sorted(word_cnt.items(), key = lambda x:int(x[1]), reverse=True)
    word_cnt = word_cnt[:config.model.top_words]

    # build up word dict
    for key, _ in word_cnt:
        word_to_id[key] = num_words
        num_words += 1
    config.model.vocab_size = len(word_to_id) + 1
    return word_to_id

In [5]:
def sent_to_tensor(batch, word_to_id):
    '''
    Inputs:
        batch: [B * T]   type:string list
    Outpus:
        tensor: [T * B]  type:tensor
    '''
    pad_len = config.model.seq_len
    batch_size = len(batch)
    
    tensor = torch.zeros(pad_len, batch_size, dtype=torch.long)
    for i in range(batch_size):
        for j in range(len(batch[i])):
            id = word_to_id.get(batch[i][j], 0)
            tensor[j][i] = id

    return tensor

In [7]:
from config import config

In [9]:
import json

In [12]:
word_to_id = build_up_word_dict()

In [59]:
test_arg1_sents, test_arg2_sents, test_labels, _ = load_PDTB("Test")

In [15]:
batch_size = config.training.batch_size

In [19]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
choise = "cuda" if torch.cuda.is_available() else "cpu"
print(choise + " is available")
device = torch.device(choise)

cuda is available


In [44]:
import numpy as np

In [49]:
from sklearn.metrics import classification_report

In [60]:
with torch.no_grad():
    result = []
    for i in range(0, len(test_arg1_sents), batch_size):
        arg1 = test_arg1_sents[i:i+batch_size]
        arg2 = test_arg2_sents[i:i+batch_size]
        label = test_labels[i:i+batch_size]

        arg1 = sent_to_tensor(arg1, word_to_id).to(device)
        arg2 = sent_to_tensor(arg2, word_to_id).to(device)

        output_Comparison = Comparison_model(arg1, arg2).transpose(0, 1)[1].cpu().numpy()
        output_Contingency = Contingency_model(arg1, arg2).transpose(0, 1)[1].cpu().numpy()
        output_Expansion = Expansion_model(arg1, arg2).transpose(0, 1)[1].cpu().numpy()
        output_Temporal = Temporal_model(arg1, arg2).transpose(0, 1)[1].cpu().numpy()
        
        
        for j in range(batch_size):
            try:
                x = np.argmax([output_Comparison[j], output_Contingency[j],output_Expansion[j], output_Temporal[j]])
                result.append(x)
            except IndexError:
                continue
    print(result)
    print(test_labels)
    print(classification_report(test_labels, result))
     
    
        
        
        
        
        

/home/zyc/jupyter/baseline/model.py:65: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  outputs1, hidden1 = self.lstm_1(embeded_1, None)
/home/zyc/jupyter/baseline/model.py:66: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  outputs2, hidden2 = self.lstm_2(embeded_2, None)


[0, 1, 2, 2, 2, 2, 2, 3, 2, 2, 0, 2, 1, 0, 2, 2, 2, 2, 1, 2, 0, 2, 2, 3, 1, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 0, 2, 0, 1, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 3, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 1, 2, 2, 0, 2, 2, 3, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 1, 2, 1, 1, 2, 0, 0, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 3, 2, 3, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 1, 0, 2, 2, 2, 2, 1, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1, 2, 3, 2, 2, 2, 2, 0, 2, 2, 0, 0, 3, 1, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 